In [5]:
import requests
import json
import pandas as pd
import numpy as np
import datetime

In [6]:
def prettify(obj):
    return json.dumps(obj, sort_keys=True, indent=2)

In [7]:
username = input('Enter username: ')
password = input('Enter password: ')
headers = {'Content-Type': 'application/json'}

In [8]:
with open ("api_key.json") as f:
    api_key = json.load(f)["mitchell"]

In [9]:
def get_data(name, start_time, end_time, interval):
    base_url = 'https://itsnt2259.iowa.uiowa.edu/piwebapi/search/query?q=name:'
    url = base_url + name
    query = requests.get(url, auth=(username, password), headers=headers).json()
    self_url = query['Items'][0]['Links']['Self']
    point = requests.get(self_url, auth=(username, password), headers=headers).json()
    data_url = point['Links']['InterpolatedData']
    data_url = data_url + '/?startTime=-' + start_time + '&endTime=-' + end_time + '&interval=' + interval
    data = requests.get(data_url, auth=(username, password), headers=headers).json()
    return data

In [10]:
# EXAMPLE USAGE OF get_data()
data = get_data('PP_Electric_Gen', '2w', '1w', '1d')

In [11]:
# Main Natural Gas
# Names
main_ng_names = ['PP_TB1_2_TB1_GAS_FLOW', 'HBLR_GAS_FLOW', 'PP_B7_Gas_Flow_Adj', 'PP_B8_Gas_Flow_Adj', 'PP_B10_FLT_235_FT', 'PP_BLR12_FT_006_KSCFH',
                 'PP_GG1_FUEL_FLOW', 'PP_GG2_FUEL_FLOW', 'PP_GG3_FUEL_FLOW', 'PP_GG4_FUEL_FLOW', 'PP_AF-XI-8220A']

# Main Purch El
main_purch_el_names = ['PP_Electric_Purch']

# Main Gen El
main_gen_el_names = ['PP_Electric_Gen']

# Blr 10 Pellets
blr_10_pellets_names = ['PP_CHS_B10WeighBelt_MvgAvg']

# Blr 11 Coal + Pellets
blr_11_coal_pellets_names = ['PP_SF-WIT-6044A']

# Blr 11 Oat Hulls
blr_11_oat_hulls_names = ['PP_BIO_Weight']

ALL_DATA = {'Main Natural Gas': main_ng_names, 'Main Purch El': main_purch_el_names, 'Main Gen El': main_gen_el_names,
            'Blr 10 Pellets': blr_10_pellets_names, 'Blr 11 Coal + Pellets': blr_11_coal_pellets_names, 'Blr 11 Oat Hulls': blr_11_oat_hulls_names}

# TODO Oakdale Nat Gas Est. using steam and NG Monthly BTU Factors
# TODO Oakdale Purch El
# TODO Oakdale Gen El

In [12]:
# Get the data
interval = '1h'
startTime = '1d'
endTime = '1h'

def get_df(names, startTime, endTime, interval):
    df = pd.DataFrame()
    for i, name in enumerate(names):
        data = get_data(name, startTime, endTime, interval)
        items = data['Items']
        if i == 0:
            times = [datetime.datetime.strptime(item['Timestamp'][:-2], '%Y-%m-%dT%H:%M:%S.%f') for item in items]
            df['Time'] = times
        vals = [max(item['Value'], 0) for item in items]
        df[name] = pd.DataFrame(vals)
    return df

In [ ]:
main_ng_df = get_df(main_ng_names, startTime, endTime, interval)
main_purch_el_df = get_df(main_purch_el_names, startTime, endTime, interval)
main_gen_el_df = get_df(main_gen_el_names, startTime, endTime, interval)
blr_10_pellets_df = get_df(blr_10_pellets_names, startTime, endTime, interval)
blr_11_coal_pellets_df = get_df(blr_11_coal_pellets_names, startTime, endTime, interval)
blr_11_oat_hulls_df = get_df(blr_11_oat_hulls_names, startTime, endTime, interval)

In [14]:
# CO2 Emission Calculations
HEAT_CONVERSION_COEFFS = { 'natural_gas': 1.026, 'pellets': 20.89375, 'oat_hulls': 8.25, 'coal': 24.93 }
CO2_CONVERSION_COEFFS = {  'natural_gas': 54.16, 'pellets': 136.025, 'oat_hulls': 154.37, 'coal': 105.88}
PURCH_CO2_COEFF = 611.169

In [15]:
# add columns 1,2,3,4,6,11 of main_ng_df to main_ng_df['Sum']
main_ng_df['Sum'] = main_ng_df.iloc[:, 1:5].sum(axis=1) + main_ng_df.iloc[:, 6:12].sum(axis=1)
# add columns 5,7,8,9,10 of main_ng_df to main_ng_df['CO2']
main_ng_df['Sum'] += (main_ng_df.iloc[:, 5:6].sum(axis=1) + main_ng_df.iloc[:, 7:11].sum(axis=1))/1000

main_ng_df['CO2'] = main_ng_df['Sum'] * HEAT_CONVERSION_COEFFS['natural_gas'] * CO2_CONVERSION_COEFFS['natural_gas']
print("Natural Gas", main_ng_df['CO2'].head())

Natural Gas 0     177.454505
1    1586.324767
2    2125.334126
3    2127.105293
4    1868.117882
Name: CO2, dtype: float64


In [16]:
# calculate CO2 emissions from blr_11_oat_hulls_df
blr_11_oat_hulls_df['CO2'] = blr_11_oat_hulls_df['PP_BIO_Weight'] * HEAT_CONVERSION_COEFFS['oat_hulls'] * CO2_CONVERSION_COEFFS['oat_hulls']
print("Oat Hulls", blr_11_oat_hulls_df['CO2'].head())

Oat Hulls 0    24892.443643
1    23073.103469
2    22832.063918
3    23148.163851
4    22796.173935
Name: CO2, dtype: float64


In [17]:
# calculate CO2 emissions from blr_10_pellets_df
blr_10_pellets_df['CO2'] = blr_10_pellets_df['PP_CHS_B10WeighBelt_MvgAvg'] * HEAT_CONVERSION_COEFFS['pellets'] * CO2_CONVERSION_COEFFS['pellets']
blr_10_pellets_df['CO2'].head()

0    21751.240795
1    24130.313975
2    22898.605294
3    20672.279292
4    20125.382153
Name: CO2, dtype: float64

In [18]:
# calculate CO2 emissions from main_purch_el_df
main_purch_el_df['CO2'] = main_purch_el_df['PP_Electric_Purch'] * PURCH_CO2_COEFF
main_purch_el_df['CO2'].head()

0    22448.720194
1    20331.445654
2    19021.511369
3    18230.103436
4    18055.609124
Name: CO2, dtype: float64

In [20]:
CURRENT_PELLET_PERCENT = 0.195
# calculate CO2 emissions from blr_11_coal_pellets_df
blr_11_coal_pellets_df['CO2'] = (
    blr_11_coal_pellets_df['PP_SF-WIT-6044A'] * CURRENT_PELLET_PERCENT * CO2_CONVERSION_COEFFS['pellets']
    + blr_11_coal_pellets_df['PP_SF-WIT-6044A'] * (1 - CURRENT_PELLET_PERCENT) * CO2_CONVERSION_COEFFS['coal'])

blr_11_coal_pellets_df['CO2'].head()

0    725.759725
1    703.245874
2    693.847616
3    703.961087
4    700.117031
Name: CO2, dtype: float64